<a href="https://colab.research.google.com/github/Hyuncastl/MACHINE_LEARNING/blob/main/12_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import shutil
source_file = "/content/archive.zip"

extract_folder = "content/"

shutil.unpack_archive(source_file, extract_folder)

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [5]:
trian_dir = "/content/content/training_set/training_set"
vaild_dir = "/content/content/test_set/test_set"

In [6]:
image_gen_aug = ImageDataGenerator(rescale = 1/255.,
                                   horizontal_flip = True,
                                   rotation_range = 35,
                                   zoom_range = 0.2
                                  )
train_gen = image_gen_aug.flow_from_directory(trian_dir,
                                              batch_size = 32,
                                              target_size = (224,224),
                                              classes = ['cats','dogs'],
                                              class_mode = 'binary')
valid_gen = image_gen_aug.flow_from_directory(vaild_dir,
                                              batch_size = 32,
                                              target_size = (224,224),
                                              classes = ['cats','dogs'],
                                              class_mode = 'binary')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [7]:
 def solution_model():
    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy']
                  )

    checkpoint_path = 'temp_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 save_best_only=True,
                                 monitor='val_loss',
                                 verbose=1
                                 )
    model.fit(train_gen, validation_data=valid_gen,
              epochs=10, callbacks=[checkpoint])

    model.load_weights(checkpoint_path)
    return model

In [ ]:
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-cats-vs-dogs.h5")

Epoch 1/10
229/251 [==========================>...] - ETA: 1:07 - loss: 23.0185 - accuracy: 0.4977